# Import libraries

In [1]:
import pandas as pd
import numpy as np
import os
import config
from joblib import dump, load
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import balanced_accuracy_score, accuracy_score
from GMHI import GMHI

# Load data

In [2]:
X = pd.read_csv(os.path.join(config.CLEAN_DIR, "taxonomic_features.csv"), index_col=[0, 1])
y = pd.read_csv(os.path.join(config.CLEAN_DIR, "is_healthy.csv"), index_col=[0, 1])
display(X.head())
display(y.head())

k__Archaea  k__Archaea|p__Euryarchaeota  \
Study_ID Sample Accession                                            
GMHI-23  SAMEA3879547        0.019774                     0.019774   
         SAMEA3879551        0.000000                     0.000000   
         SAMEA3879543        0.003253                     0.003253   
         SAMEA3879565        0.008863                     0.008863   
         SAMEA3879546        0.000048                     0.000048   

                           k__Archaea|p__Euryarchaeota|c__Methanobacteria  \
Study_ID Sample Accession                                                   
GMHI-23  SAMEA3879547                                            0.019774   
         SAMEA3879551                                            0.000000   
         SAMEA3879543                                            0.003253   
         SAMEA3879565                                            0.008863   
         SAMEA3879546                                            0.000048   

                           k__Archaea|p__Euryarchaeota|c__Methanobacteria|o__Methanobacteriales  \
Study_ID Sample Accession                                                                         
GMHI-23  SAMEA3879547                                               0.019774                      
         SAMEA3879551                                               0.000000                      
         SAMEA3879543                                               0.003253                      
         SAMEA3879565                                               0.008863                      
         SAMEA3879546                                               0.000048                      

                           k__Archaea|p__Euryarchaeota|c__Methanobacteria|o__Methanobacteriales|f__Methanobacteriaceae  \
Study_ID Sample Accession                                                                                                
GMHI-23  SAMEA3879547                                               0.019774                                             
         SAMEA3879551                                               0.000000                                             
         SAMEA3879543                                               0.003253                                             
         SAMEA3879565                                               0.008863                                             
         SAMEA3879546                                               0.000048                                             

                           k__Archaea|p__Euryarchaeota|c__Methanobacteria|o__Methanobacteriales|f__Methanobacteriaceae|g__Methanobrevibacter  \
Study_ID Sample Accession                                                                                                                      
GMHI-23  SAMEA3879547                                               0.019774                                                                   
         SAMEA3879551                                               0.000000                                                                   
         SAMEA3879543                                               0.003253                                                                   
         SAMEA3879565                                               0.008863                                                                   
         SAMEA3879546                                               0.000048                                                                   

                           k__Archaea|p__Euryarchaeota|c__Methanobacteria|o__Methanobacteriales|f__Methanobacteriaceae|g__Methanobrevibacter|s__Methanobrevibacter_smithii  \
Study_ID Sample Accession                                                                                                                                                    
GMHI-23  SAMEA3879547                                               0.019774              

PHENOTYPE:Healthy_Nonhealthy
Study_ID Sample Accession                              
GMHI-23  SAMEA3879547                              True
         SAMEA3879551                              True
         SAMEA3879543                              True
         SAMEA3879565                              True
         SAMEA3879546                              True

# Train Models!

## Train GMHI model on entire dataset

In [8]:
gmhi = GMHI(use_shannon=True, theta_f=1.4, theta_d=0.1)

# use only non-viral species
X_GMHI = X.copy()
X_GMHI = X_GMHI[list(filter(lambda x : "s__" in x and "virus" not in x, X_GMHI.columns))]

gmhi.fit(X_GMHI, y)
GMHI_scores = pd.DataFrame(gmhi.decision_function(X_GMHI), index=y.index, columns=["GMHI"])
display(GMHI_scores)
print("GMHI balanced_accuracy on training set:", balanced_accuracy_score(y, GMHI_scores > 0))
idx = (abs(GMHI_scores) > 0.1).values
print("GMHI balanced_accuracy on training set |GMHI| > 0.1:", balanced_accuracy_score(y[idx], GMHI_scores[idx] > 0))

GMHI
Study_ID Sample Accession          
GMHI-23  SAMEA3879547      2.836720
         SAMEA3879551      1.030885
         SAMEA3879543      1.518635
         SAMEA3879565      1.229460
         SAMEA3879546      4.231553
...                             ...
P140     SAMN07509557      0.627861
         SAMN07509558      0.000000
         SAMN07509546     -2.375201
         SAMN07509552      1.190039
         SAMN07509921      0.961839

[9045 rows x 1 columns]

GMHI balanced_accuracy on training set: 0.7037794238618238
GMHI balanced_accuracy on training set |GMHI| > 0.1: 0.7099197170444944


## Train GMHI2 model on entire dataset

In [9]:
gmhi2 = LogisticRegression(random_state=42, penalty="l1", solver="liblinear", C=config.REGULARIZATION, class_weight="balanced")
X_GMHI2 = X > config.PRESENCE_CUTOFF
gmhi2.fit(X_GMHI2, y.values.flatten())
GMHI2_scores = pd.DataFrame(gmhi2.decision_function(X_GMHI2), index=y.index, columns=["GMHI2"])
display(GMHI2_scores)
print("GMHI2 balanced_accuracy on training set:", balanced_accuracy_score(y, GMHI2_scores > 0))
idx = (abs(GMHI2_scores) > 0.1).values
print("GMHI2 balanced_accuracy on training set |GMHI2| > 0.1:", balanced_accuracy_score(y[idx], GMHI2_scores[idx] > 0))


GMHI2
Study_ID Sample Accession          
GMHI-23  SAMEA3879547      1.864841
         SAMEA3879551     -0.068806
         SAMEA3879543     -0.870357
         SAMEA3879565      0.396061
         SAMEA3879546      1.148468
...                             ...
P140     SAMN07509557      0.014177
         SAMN07509558      0.000000
         SAMN07509546     -1.013114
         SAMN07509552     -0.809779
         SAMN07509921     -0.132474

[9045 rows x 1 columns]

GMHI2 balanced_accuracy on training set: 0.7901139052467988
GMHI2 balanced_accuracy on training set |GMHI2| > 0.1: 0.8056839921584207


# Export stuff

## Export GMHI scores

In [44]:
GMHI_scores.to_csv(os.path.join(config.PREDICTION_DIR, "GMHI_scores.csv"))

## Export GMHI2 scores and serialize model

In [45]:
GMHI2_scores.to_csv(os.path.join(config.PREDICTION_DIR, "GMHI2_scores.csv"))

In [46]:
dump(gmhi2, os.path.join(config.MODEL_DIR, 'gmhi2_model.joblib'))

['../output/models/gmhi2_model.joblib']